# Tokenization

[Video](https://www.youtube.com/watch?v=zduSFxRajkE)<br>
[Repository](https://github.com/karpathy/minbpe)<br>
[Eureka Labs Discord](https://discord.com/invite/3zy8kqD9Cp)

## Table of Contents

- [Flashback: Character-Level Tokenization](#flashback-character-level-tokenization)
- [The Pitfalls of Tokenization](#the-pitfalls-of-tokenization)
- [The Tokenizer Idea](#the-tokenizer-idea)
  - [Unicode](#unicode)
- [Byte-Pair Encoding](#byte-pair-encoding)
  - [Decoding](#decoding)
  - [Encoding](#encoding)
- [Going SOTA: Tokenizers in the Wild](#going-sota-tokenizers-in-the-wild)
  - [GPT Tokenizers](#gpt-tokenizers)
  - [OpenAI TikToken](#openai-tiktoken)
  - [Special Tokens](#special-tokens)
  - [Sentencepiece](#sentencepiece)
- [Exercise Time](#exercise-time)
- [Looping Back: vocab_size](#looping-back-vocab_size)
- [Conclusion](#conclusion)
- [References](#references)

Understanding the inner workings of Large Language Models (LLMs) requires a deepdrive into the very first step required for the generative process: **Tokenization**.<br><br>
**Tokenization describes the process of converting a character sequence into a sequence of numerically representative tokens.**<br>
**Tokens are the basic units of meaning for a language model.**<br><br>
LLMs are *purely mathematical* models. They are unable to process raw text directly and instead *only ever process numbers*.<br>
The task seems simple enough: Find a way to most fittingly map input text to some numeric representation that can be processed.<br><br>
If this mapping is done incorrectly, the transformation often emerges as *the* root cause for a multitude of downstream issues.

## Flashback: Character-Level Tokenization

In the [previous lecture](../N007%20-%20GPT%20From%20Scratch/N007%20-%20GPT.ipynb), we already implemented a simplified form of tokenization for our GPT model.<br>
A quick review of that process:

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import regex as re
import tiktoken
import os
import json

For a start, we loaded a sample dataset from the `tiny-shakespeare.txt` text file to memory:

In [3]:
# Read the txt file to inspect it
with open('../tiny-shakespeare.txt', 'r') as f:
    text = f.read()

# Print a sample of the text (First 100 characters)
print("Length of Dataset:", len(text), "\n")
print(text[:100])

Length of Dataset: 1115394 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


We then determined all the unique characters present in the dataset:

In [3]:
chars = sorted(list(set(text))) # Get all unique characters in the text
vocab_size = len(chars)         # Length of vocabulary (this includes the space character and newline)
print('Unique Characters:', ''.join(chars))
print(f'\nVocabulary size: {vocab_size}')

Unique Characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

Vocabulary size: 65


Given these unique characters found in the text file, we went on to map each of them to a respective integer,<br>
allowing for a fully numeric representation of the characters and thus the entire dataset:

In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }     # Character to index mapping
itos = { i:ch for i,ch in enumerate(chars) }     # Index to character mapping

# This is the encoder; Encode a string to a list of integers
encode = lambda s: [stoi[c] for c in s]
# This is the decoder; Decode a list of integers to a string
decode = lambda l: ''.join([itos[i] for i in l])

msg = "hii there"
token_list = encode(msg)
print(token_list)
print(decode(token_list))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


> The above implemented approach is called **character-level tokenization**.

Encoding the sum of unique characters in the input text file would result in an equal sum of tokens for our GPT to process.<br>
But, we didn't stop there just yet.<br>
Instead, we went on to built a look-up table to map each numeric token to a vector representation.<br>
<br>
**Why did we take this extra step?**<br>
The vocabulary representation by $65$ vectors (one per token) á $65$ dimensions (one per token, again) enabled a deeper, fixed-resolution representation for each token.<br>
We then left it up to our `BigramLM` to, per token, optimize the values inside this token's vector representation.<br>
<br>
**All in all, a uniformly sized representation of tokens by learnable vectors allowed the `BigramLM` to better capture compositional relationships between characters.**<br><br>
**Think of it like this:** Characters that are semantically or functionally similar can now attain similar vector representations.<br>
Therefore, vowels or consonants may cluster together in the vector space, which made it easier for `BigramLM` to capture similarities in associations.<br>
Vectors are a lot more expressive than discrete, integer-based representations of tokens.<br><br>The Embedding Layer `BigramLM` that achieved all of this looked like so:

In [5]:
torch.manual_seed(1337) # for reproducibility

# Not really an LM at this stage, but we will get there...
class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Embedding the vocabulary
        # Every one of the vocab_size tokens is represented by a vector of size vocab_size
        self.embed = nn.Embedding(vocab_size, vocab_size) # 65 unique 65-dim vectors

    def forward(self, idx, targets):
        # idx is of shape (batch_size, block_size)
        # targets is of shape (batch_size, block_size)
        logits = self.embed(idx)
        return logits # Embed the input indices, shape is now (batch_size, block_size, vocab_size) (B, T, C)

Even with the above described $65 \times 65$ embedding matrix, it turns out that we only implemented an inefficient tokenization at best.<br>
In reality, token vocabularies are constructed much differently, going above and beyond just the character-level and the immediate character-to-numeric-to-vector representation mapping.

> Text is actually not tokenized on character-level, but on what is called *chunk-level*.

**Our goal is to find and explore a way to tokenize text on the chunk-level instead of character-level.**

## The Pitfalls of Tokenization

Let's stay with the fundamental notion of tokens being a fundamental unit of information.<br>
The goal is to translate strings of text into representations that are most informative and interpretable for LLMs.<br>
<br>
Tokenization, if done incorrectly, can be the reason for *boatloads* of downstream issues for LLMs.<br>
Some of the most common issues related to tokenization are:

- Why can't the LLM spell?
- Why can't the LLM do simple text manipulations like reversing a string?
- Why can an LLM turn out to be worse with non-English languages?
- Why can't an LLM produce simple arithmetic operations correctly?
- Why could entering some special character like `<|endoftext|>` halt generation?
- Why can an LLM have hiccups due to 'tailing whitespaces'?
- Why can an LLM break down when encountering capital letters within a word?
- Why might working with LLMs through YAML be preferable to using JSON?
- Why does 'LLM' not actually mean 'end-to-end language modeling'?

> What's the root of *all* evil? **Tokenization!**

Let's dive into a practical example with the [GPT-2 Tokenizer](https://insightcivic.s3.us-east-1.amazonaws.com/language-models.pdf) over at [tiktokenizer.vercel.app](https://tiktokenizer.vercel.app):

```md
Tokenization is at the heart of much weirdness of LLMs. Do not brush it off.

127 + 677 = 804
1275 + 6773 = 8041

Egg.
I have an Egg.
egg.
EGG.

만나서 반가워요. 저는 OpenAI에서 개발한 대규모 언어 모델인 ChatGPT입니다. 궁금한 것이 있으시면 무엇이든 물어보세요.

for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
```
<br>

![](./img/Tiktoken_Vercel_1.png)

Each color represents a different token. It's clear that GPT-2's tokenizer is not operating on character-level.<br>
<br>
This might not have too much of an impact when working with plain text, but take a look at the tokenization of the arithmetic operations.<br>
Every numeric value except $127$ is sliced into multiple tokens.<br>
Logical mis-slicing by tokens can be problematic here, as it may lead to challenges in preserving the semantic meaning of any numerical or arithmetic expression.<br>
<br>
Curiously, the word "Egg" is tokenized in four different ways, depending on the degree of capitalization and leading space<br>
We can also see that the tokenization of Korean text is more granular compared to that of English text.<br>
This can to a great extent be attributed to representational imbalance in the training data. The tokenizer may just have become more optimized for English text.<br>
Because of this, a Korean-speaking end-user might have to pay a higher price to an LLM provider than an English-speaking user for completing the same tasks with the LLM, just because of poorer/more granular tokenization filling up the token count of the context window more quickly.<br>
Additionally, having more tokens ultimately representing the same amount of data results in a more rapidly filled attention buffer downstream.<br>
This inevitably leads to an overall decrease in the performance of the LLM.<br>
<br>
With the Python code example, each whitespace is tokenized individually. This reduces the interpretability of the code for the LLM, as the context window is rapidly spammed with whitespace tokens.<br>
Just like with the Korean text, this will negatively impact model performance.

When providing the same input to the GPT-4 tokenizer, `cl100k_base`, we pretty much half the token count.<br>
This implies the tokenizer has approximately a twice as large vocabulary as the GPT-2 tokenizer:

![](./img/cl100k_base_1.png)

> The increase in vocabulary size *alone* enables GPT-4 to pretty much double the context window size, compared to GPT-2.

## The Tokenizer Idea

> The tokenizer is a separate entity from the LLM.<br>It can be tuned or trained on specific text independently, while the LLM is optimized based on different text.<br>And yet, crucially, while the tokenizer doesn't rely on the LLM, it serves as an interface between the LLM and the text.<br>The LLM therefore is enabled to operate in a purely tokenized world.

![](./img/Tokenizer_Schema.png)

We now want to create a Tokenizer that goes beyond character-level tokenization and represents some chunks by some identifying value.<br>
From there, we can go on and build a look-up table to map each chunk-representing token to a vector representation. This step remains as before.

Oh, and before we forget, this time the tokenizer should be able to handle different scriptural systems, too. And Emojis, *we need Emoji support!*

### Unicode

Python can handle different scriptural systems out-of-the-box:

In [4]:
some_text = "안녕하세요 👋 (hello in Korean!)"
print(some_text)

안녕하세요 👋 (hello in Korean!)


The [documentation](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str) states that Python's `str` type is a **sequence of [Unicode](https://en.wikipedia.org/wiki/Unicode) codepoints**.<br>
Unicode is a character encoding standard, mapping each character to a unique integer value.<br><br>
We can read this value by using the `ord()` function:

In [5]:
print([ord(x) for x in some_text])

[50504, 45397, 54616, 49464, 50836, 32, 128075, 32, 40, 104, 101, 108, 108, 111, 32, 105, 110, 32, 75, 111, 114, 101, 97, 110, 33, 41]


So that's it, we solved our encoding problem, ... right?<br>
**Well, not really.**<br>
<br>
While Unicode is comprehensive, it is changing, evolving, and already pretty vast. **Unicode is not stable.**<br>
But, Unicode has defined three encoding forms which *are* stable and can be used to represent<br>a subset of Unicode characters: [UTF-8](https://en.wikipedia.org/wiki/UTF-8), [UTF-16](https://en.wikipedia.org/wiki/UTF-16), and [UTF-32](https://en.wikipedia.org/wiki/UTF-32).<br>
Character representation through these encodings is achieved with sequences of bytes.<br>

> **Why can we see from the code above that Unicode represents characters by integer, but UTF-8, UTF-16, and UTF-32 use bytes?**<br>
> UTF-8, UTF-16, and UTF-32 are encoding schemes, defining how Unicode is represented in binary form (sequences of bytes) for storage and transmission.<br>
> For example, UTF-8 is specifically designed to handle the evolving nature of the Unicode standard while remaining stable itself.<br>
> Unicode assigns integer values to characters, while UTF-8, UTF-16, and UTF-32 are underlying encoding schemes that describe how these code points are transformed into sequences of bytes for storage and transmission.

With UTF-8, you predictably can represent Unicode characters by a sequence of $1$ to $4$ bytes. **This is fixed.**<br>
You may refer to [Nathan Reed's Blogpost on Unicode](https://www.reedbeta.com/blog/programmers-intro-to-unicode/) for further details, especially for the [UTF-8 Everywhere Manifesto](https://utf8everywhere.org/).

The following is the set of raw bytes representing our string according to UTF-8:

In [7]:
list(some_text.encode('utf-8'))

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

Still, we don't want to use the raw bytes as tokens. **But why not?**<br>
<br>
You can see above that we only distinguish between $256$ different representations, because we work with $8$-bit-representations/bytes.<br>
While, yes, we could use this byte-by-byte representation to represent our byte array, it would be a very inefficient way to represent the encoded text.<br>
We would essentially zoom into the text and differentiate parts of it (byte-parts of letter encodings) on a very fine level, thereby avoidably stretching out the token count.<br>
And this, in turn, would clog up the attention buffer of the LLM, limiting downstream capabilities.

> We want to support a large vocabulary, using UTF-8 as a basis for tokenization, but **we don't want to use the raw bytes as tokens, as this would be inefficient**.

(We should note that whatever we do from here on will create some sort of overhead or abstraction on top of UTF-8. In a perfect world, this would be avoided. Interestingly, efforts like [\[Yu et al., 2023\]](https://arxiv.org/abs/2305.07185) are trying to do just that: Avoiding tokenization overhead and getting closer to using raw bytes as tokens. Proof is still pending, though.)

## Byte-Pair Encoding

The [Wikipedia article on Byte Pair Encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) is quite hands-on and definitely recommended.<br>

> Iteratively, for a given text, BPE merges the most frequent pair of consecutive tokens into a single new token, which we then add to our vocabulary.<br> We repeat this until we reach a predefined vocabulary size.<br>

You can see how this can build up token hierarchies, as the most frequent pairs are merged first, and then the next most frequent pairs, and so on.<br>
<br>
**Let's see how this works in practice:**

In [8]:
# Text is the first paragraph from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode('utf-8')   # Byte array
tokens = list(map(int, tokens)) # Convert to list of integers for visualization

print(text, "\n")
print("Original Text Length:", len(text), "\n\n")
print(tokens, "\n")
print("Length of Token List:", len(tokens)) # Simple characters map to one byte, but e.g. emojis map to 4 bytes

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception. 

Original Text Length: 533 


[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 22

In [9]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Sliding Window of size 2 across tokens 
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(sorted(((v,k) for k,v in stats.items()), reverse=True)) # Invert key-value relationship and sort by key (count)
print(sorted(((v,(chr(k[0]), chr(k[1]))) for k,v in stats.items()), reverse=True)[:5]) # Just for fun, 5 most common bigrams written out

top_pair = max(stats, key=stats.get) # Retrieve the most common bigram
print(top_pair)

[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

The token vocabulary at this point spans $0$ to $255$ because we are working with $8$-bit-representations/bytes.<br>
We can now create a new, $256^{\text{th}}$ token representing the most common pair, `('e', ' ')`:

In [10]:
def merge(ids, pair, idx):
    # Iterating through ids, if we find (pair), replace it by value idx
    newids = []
    i = 0
    while i < len(ids):
        # If we are not at the very last position AND the pair matches, replace it
        if i < len(ids)-1 and (ids[i], ids[i+1]) == pair:
            newids.append(idx)
            i += 2 # We skip over the now replaced pair
        else:
            newids.append(ids[i])
            i += 1
    return newids

# Sanity-Check
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

[5, 6, 99, 9, 1]


In [11]:
tokens2 = merge(tokens, top_pair, 256)

print(tokens2, "\n")
print("Length of Token List:", len(tokens2))
print("All Occurrences Removed" if (True if top_pair not in zip(tokens2, tokens2[1:]) else False) else "Still Some Occurrences Left")

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 226, 128, 156

We effectively mapped our text to UTF-8, identified the most common pair of consecutive byte values (ranging from $0$ to $255$ in $8$-bit values), and replaced its occurences with a new token ($256$) representing the corresponding pair. **That's all we did.**<br>
We did not keep track of this action through a vocabulary or anything.

Now that we made sure our basic implementation works, we can loop over the tokens as many times as we want to actually build up a vocabulary.<br>
You can think of this vocabulary enrichment as iteratively building a binary tree from the leaves down to the root.<br>
<br>
We will use the entire blog post for training:

In [12]:
text = """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

We can now add the vocabulary as a simple dictionary called `merges` that uses the pair as key and the representing token as value:

In [13]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Sliding Window of size 2 across tokens 
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge(ids, pair, idx):
    # Iterating through ids, if we find (pair), replace it by value idx
    newids = []
    i = 0
    while i < len(ids):
        # If we are not at the very last position AND the pair matches, replace it
        if i < len(ids)-1 and (ids[i], ids[i+1]) == pair:
            newids.append(idx)
            i += 2 # We skip over the now replaced pair
        else:
            newids.append(ids[i])
            i += 1
    return newids


vocab_size = 276    # Our (Arbitrary) Target Vocabulary Size
num_merges = vocab_size - 256
ids = list(tokens)  # We'll work on a copy of tokens from here on (list() makes a deep copy of a list)
merges = {}         # (int, int) -> int, Merge dictionary; Think of this as key: (child1, child2), value: parent/new token


for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get) # Retrieve the most common bigram
    idx = 256 + i
    print(f"Merging {pair}\tinto new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

Merging (101, 32)	into new token 256
Merging (105, 110)	into new token 257
Merging (115, 32)	into new token 258
Merging (116, 104)	into new token 259
Merging (101, 114)	into new token 260
Merging (99, 111)	into new token 261
Merging (116, 32)	into new token 262
Merging (226, 128)	into new token 263
Merging (44, 32)	into new token 264
Merging (97, 110)	into new token 265
Merging (111, 114)	into new token 266
Merging (100, 32)	into new token 267
Merging (97, 114)	into new token 268
Merging (101, 110)	into new token 269
Merging (257, 103)	into new token 270
Merging (261, 100)	into new token 271
Merging (121, 32)	into new token 272
Merging (46, 32)	into new token 273
Merging (97, 108)	into new token 274
Merging (259, 256)	into new token 275


Comparing the enriched vocabulary to the original one, we can see that we now need fewer tokens to represent the same amount of data because we are able to represent more complex chunks of text with a single respective token:

In [14]:
print("Old Length of Token List:", len(tokens))
print("New Length of Token List:", len(ids))
print(f"Compression Ratio: {len(tokens) / len(ids):.2f}x")

Old Length of Token List: 24597
New Length of Token List: 19438
Compression Ratio: 1.27x


Only $20$ merges and token creations were necessary to achieve a compression of the vocabulary by a factor of $1.27$.<br><br>
Thanks to `merges`, we can do both encoding and decoding.<br>The tokenizer has become a translation layer between LLM and readable text.

But, again, token depth is not the only parameter worth optimizing. The dataset from which we derive the byte-pairs should be very well representative of the full range of text you want to tokenize (at least for your intended use cases). And we saw the implications of this choice of training data already with the GPT-2 vs. GPT-4 tokenizer comparison.

### Decoding

With encoding done, given a sequence of integer tokens within range $[0;\ \text{vocab\_size}]$, what is the text string?

In [15]:
# Decoder Preprocessing: Mapping from token-id to bytes-object for that token
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():   # This needs to run in the order in which we inserted items into merges (use Python >= 3.7)
    vocab[idx] = vocab[p0] + vocab[p1] # Populate at idx (parent integer) with concatenated bytes-object of children p0 and p1, format is {idx: b'p0p1'}

def decode(ids):
    # Given ids (list of integers), return the Python string
    tokens = b"".join(vocab[idx] for idx in ids) # Concatenate all the bytes-objects for each new token-id
    text = tokens.decode("utf-8")                # Decode the bytes into a Python string
    return text

**Ok, what did we just do?**<br>
<br>
The dictionary `vocab` in a first step is made to hold byte-objects as values to the respective integer keys $0$ to $255$ as keys.<br>
In addition to this 'base-mapping' of `int -> byte(int)`, we now want to store the `new_token` as the key and `(token1, token2)` as the value.<br>
You can think of it as reversing the key-value relationship from `(token1, token2) -> new_token` in `merges` to now be `new_token -> (token1, token2)` in `vocab`.<br>
However, `vocab` will not store `(token1, token2)` as a tuple. Instead, it will store the concatenation of the two bytes-objects, hence `vocab[p0] + vocab[p1]` or `byte(token1) + byte(token2)`.

> `vocab` now holds the mapping for every `token` (not only `new_token`, but all tokens) integer to either a single byte, or the concatenation of the two bytes-objects that were merged to create `new_token`.

The `decode` function then goes to receive a list of token integers and iteratively indexes into `vocab` at the respective token integer to step-by-step join together the byte-objects to form a string.<br>
Finally, UTF-8 decoding is used to convert the byte-objects to something more readable, ideally closely resembling the original text.<br>
<br>
You might have noticed that this still has an implementation issue left, though:

In [16]:
print(decode([97]))  # Works out to be "a"
print(decode([128])) # Works out to be ... flawed?!

a


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

Mapping $128_{10}$ to binary, we get $1000\ 0000_{2}$, which is not a valid UTF-8 start byte as per [Wikipedia](https://en.wikipedia.org/wiki/UTF-8#Encoding).<br>
We provide $1000\ 0000_{2}$, but only either $0\text{XXX\ XXXX}_{2}$ or $110\text{X\ XXXX}_{2}$ start a valid UTF-8 byte representation.<br>
<br>
**Luckily, we can fix this pretty easily:**

In [17]:
# Decoder Preprocessing: Mapping from token-id to bytes-object for that token
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():   # This needs to run in the order in which we inserted items into merges (use Python >= 3.7)
    vocab[idx] = vocab[p0] + vocab[p1] # Populate at idx (parent integer) with concatenated bytes-object of children p0 and p1

def decode(ids):
    # Given ids (list of integers), return the Python string
    tokens = b"".join(vocab[idx] for idx in ids)     # Concatenate all the bytes-objects for each new token-id
    text = tokens.decode("utf-8", errors="replace")  # Decode the bytes into a Python string
    return text

In [18]:
print(decode([97]))  # Works still
print(decode([128])) # Works now

a
�


**Why is `errors=replace` effective?**<br>
<br>
See the [Python Documentation:](https://docs.python.org/3/library/codecs.html#error-handlers) If the decoder encounters some byte sequence that can't be decoded into valid Unicode, the sequence will be replaced with a special Unicode replacement character (U+FFFD). We basically take a graceful step over the invalid byte sequence, exclaim *"I don't know what this is"*, and move on.<br>
<br>
**If we just take a step over this issue now, how can this be a problem at all in the first place? Didn't we only input UTF-8 into the encoder? How could something else then return?**<br>
<br>
Well yes, we input UTF-8 into the encoder. But, in reality, we don't decode what we just tokenized. Decoding is done to the LLM output.<br>
And there, if you encounter an LLM output symbol of $128$, something went wrong on the LLM-side that is worth investing further training to avoid.<br>
<br>
**It feels like this implementation of `decode` is too shallow.**<br>
**How come this setup can consider that some tuples inside `merges` are themselves made up of `new_token`s?**<br>
**Wouldn't that give an error?**<br>
<br>
This implementation seems shallow only at first. It really isn't. We progressively build up `vocab` in the exact order that `new_token`s were added to `merges`. This way, the tuples represented by the `new_token` are guaranteed to already be in `vocab`, ready to be referenced. And if you reference a `new_token` already in `vocab`, you will get the concatenation of the bytes-objects that were merged to create `new_token`. The bytes-objects are recursively built up in `vocab`. So, if a `new_token` is found to represent a tuple of `new_token`s, the very underlying bytes-objects are found through recursive indexing into `vocab`, automatically. This makes the loop around `vocab[idx] = vocab[p0] + vocab[p1]` a very elegant solution to finding the actual byte-objects that represent each `new_token`.

### Encoding

Given a text string, what is the sequence of integer tokens $0$ to $\text{vocab\_size} - 1$?

In [19]:
def encode(text):
    tokens = list(text.encode("utf-8")) # raw bytes formatted as integer list
    # Now, lookup into merges (historically accurate from top to bottom) and replace the pair with the new token-id recursively
    while True:
        stats = get_stats(tokens) # Count how many times each pair occurs, format: (int, int) -> int (occurrence count)
        pair = min(stats, key=lambda p: merges.get(p, float('inf'))) # Iterating over keys of stats here; Retrieve the pair with the lowest merge index
        if pair not in merges: # This could be because merges doesn't contain any pair that occurs in tokens
            break # We can stop, nothing more to merge here
        idx = merges[pair] # Retrieve new token-representation for mergable pair
        tokens = merge(tokens, pair, idx) # Every pair is replaced with idx, just as we did earlier
    return tokens

With `get_stats`, we curiously don't really care about the occurrent counts. Much rather, the keys in `get_stats` form a list of worth-considering token pairs for merge.<br>
From a `merges` perspective, we want to do the merges that are listed the earliest inside `merges` first.<br>
Then, for any `pair` inside `stats`, we look into our `merges` dictionary. More specifically, we look at the `new_token` that `pair` would create. We do so with the aim of finding the `new_token` that is smallest in value.<br>
<br>
For example, let's say that the first entry in `merges` is `(1, 2) -> 256`.<br>
If this combination occurs in the `tokens` and is considered worth tokenizing by `get_stats`, we reach into `merges`.<br>
There we look for the smallest possible value that can be attributed to a pair (here it's `256` for `(1, 2)`) and retrieve it as value of `pair`.<br>
Because of this, we determined `(1, 2)` as both actually present in the text to be encoded and we deemed it most eligible for tokenization, as it's `new_token` is a smallest.<br>
<br>
With this implemented, let's test it a bit:

In [20]:
print(encode("hello world!"))
print(encode("h"))

[104, 101, 108, 108, 111, 32, 119, 266, 108, 100, 33]


ValueError: min() arg is an empty sequence

With only a single character or an empty string, we can't run `stats = get_stats(tokens)`, because there are no pairs to be found.<br>
<br>
*Let's fix this:*

In [21]:
def encode(text):
    tokens = list(text.encode("utf-8")) # raw bytes formatted as integer list
    # Now, lookup into merges (historically accurate from top to bottom) and replace the pair with the new token-id recursively
    while len(tokens) > 1:
        stats = get_stats(tokens) # Count how many times each pair occurs, format: (int, int) -> int (occurrence count)
        pair = min(stats, key=lambda p: merges.get(p, float('inf'))) # Iterating over keys of stats here; Retrieve the pair with the lowest merge index
        if pair not in merges: # This could be because merges doesn't contain any pair that occurs in tokens
            break # We can stop, nothing more to merge here
        idx = merges[pair] # Retrieve new token-representation for mergable pair
        tokens = merge(tokens, pair, idx) # Every pair is replaced with idx, just as we did earlier
    return tokens

print(encode("hello world!"))
print(encode("h"))
print(encode(""))

[104, 101, 108, 108, 111, 32, 119, 266, 108, 100, 33]
[104]
[]


Finally, let's decode something encoded:

In [22]:
print(decode(encode("hello world!"))) # Works

# We know this text already, the tokenizer was trained on it
text2 = decode(encode(text))
print('Training Text is Reconstructable: ', text2 == text) # Works

# We don't know this text, it's not in the training data
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print('Unseen Text is Reconstructable:   ', valtext2 == valtext) # Works

hello world!
Training Text is Reconstructable:  True
Unseen Text is Reconstructable:    True


**Does this `decode(encode(x))` yield `x` for every instance of `x`?**<br>
<br>
No, not always. We can achieve this quality of encoding when working with pure UTF-8 text.<br>However, when working with mixed text, we can't guarantee $1:1$ reconstruction.<br>
But, by any means, we just made a large step towards a more efficient tokenization than character-level tokenization.

## Going SOTA: Tokenizers in the Wild

### GPT Tokenizers

As per [\[Radford et al., 2019\]](https://insightcivic.s3.us-east-1.amazonaws.com/language-models.pdf), GPT-2 was the first GPT version to motivate the use of byte-pair encoding [\[Sennrich et al., 2015\]](https://arxiv.org/abs/1508.07909) for tokenization.<br>
Interestingly, GPT-2 implemented BPE in large parts just like we did. But then the paper deviates.<br><br>
Imagine a word that appears very frequently in a dataset. Each time it occurs, it may be surrounded by different other words/characters.<br>
Depending on how the tokenizer processes this, several things could happen:
- The tokenizer might decide to split or merge the word together with parts of its surrounding context if those parts co-occur often.
- If parts of the word change depending on the context, the tokenizer might create multiple tokens for parts of the same word.
- The tokenizer could even start associating parts of the word with other frequent leading or trailing characters from the context.

As the paper points out, this behavior is suboptimal. It results in tokens that reflect statistical frequency rather than genuine contextual meaning, leading to clusters that look abundant but aren’t actually semantically relevant.
<br>
To solve this, some types of characters are enforced to never be merged.<br>
Refer to [GitHub - OpenAI/gpt-2/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py) for the actual tokenizer implementation.<br>
Specifically, refer to line $53$. This holds a regex pattern that is used to rule out characters from merging together.<br>

In [3]:
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""") # |: OR operator

# 's, 't, 're, 've, 'm, 'll, 'd are all contractions (curiously case-sensitive) to be singled out
# " ?\p{L}+" matches any sequence of letters with an optional leading space
# " ?\p{N}+" matches any sequence of digits with an optional leading space
# " ?[^\s\p{L}\p{N}]+" matches any sequence of non-whitespace, non-letter, non-digit characters with an optional leading space
# "\s+(?!\S)" matches any sequence of whitespace characters that are not followed by a non-whitespace character
# "\s+" matches any sequence of whitespace characters

print(re.findall(gpt2pat, "Hello world how are you? I've heard you're      4.543 billion years old??!")) # Works, """ ?\p{L}+""" matches "Hello", """ ?\p{L}+""" matches "world" etc.

['Hello', ' world', ' how', ' are', ' you', '?', ' I', "'ve", ' heard', ' you', "'re", '     ', ' 4', '.', '543', ' billion', ' years', ' old', '??!']


Instead of directly encoding the input as-is, GPT-2 splits it into a list of regex-compliant input text chunks.<br>
The list entries are processed individually, then token representations are concatenated together.<br>

> You are effectively limited to only ever finding tokens within the bounds of regex-compliant chunks, not across them.

Funny enough, `r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""` is quite suboptimal.<br>
Think of all capital `"SHOULD'VE TESTED THAT"`.<br>
The above rules for `'ve` don't match, because they are case-sensitive.

In [24]:
print(re.findall(gpt2pat, "SHOULD'VE TESTED THAT"))
print()

example = """for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
"""
print(re.findall(gpt2pat, example))

['SHOULD', "'", 'VE', ' TESTED', ' THAT']

['for', ' i', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n   ', ' else', ':', '\n       ', ' print', '(', 'i', ')', '\n']


One could assume that BPE can now be run on these input text chunks directly, but this not the case.<br>
But, curiously, take a look at the output of the regex-compliant chunks for the code example and compare that to what [TikTokenizer](https://tiktokenizer.vercel.app) outputs:

![](./img/Tiktoken_Vercel_1.png)

**Something is off.**<br>
OpenAI seems to put an additional chunking and slicing step in between the regex-compliant chunks and the text pieces ready for BPE tokenization.<br>
What's the reason for this? We can only speculate at this point.

### OpenAI TikToken

TikToken is the [official OpenAI tokenizer implementation](https://github.com/openai/tiktoken).<br>
It delivers an interface to the tokenizers used in GPT-2, GPT-3, GPT-4 and GPT-5:

In [25]:
# GPT-2 Tokenizer
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    Hello World?!!")) # whitespace remains unmerged, as seen on TikTokenizer.vercel.app

# GPT-4 Tokenizer
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    Hello World?!!"))

[220, 220, 220, 18435, 2159, 30, 3228]
[262, 22691, 4435, 30, 3001]


[See this file](https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py) in the OpenAI TikToken repository.<br><br>
For the GPT-2 tokenization, we can see that `r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""` is functionally equivalent to [GitHub - OpenAI/gpt-2/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)<br><br>
For the GPT-4 tokenization, we can also see that this pattern was improved to `r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""`.<br>
<br>
Here's the rundown of this regex pattern:

In [ ]:
gpt4pat = re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""") 

# | is the logical OR operator

# (?i:[sdmt]|ll|ve|re)       matches 's, 'd, 'm, 't, 'll, 've, 're *non case-sensitive* now
# [^\r\n\p{L}\p{N}]?+\p{L}+  matches letter sequences with optional leading non-letter, non-digit characters
# \p{N}{1,3}                 matches digit sequences of length 1 to 3 (interesting! prevents long number tokens)
#  ?[^\s\p{L}\p{N}]++[\r\n]* matches optional spaced followed by one or more non-space, non-letter, non-number characters, then allows for however many newlines
# \s*[\r\n]                  matches none or more whitespace characters, followed by a single newline character
# \s+(?!\S)                  matches one or more whitespaces that are not followed by a non-whitespace character
# \s+                        matches one or more whitespace characters (no negative lookahead)

print(re.findall(gpt4pat, "P. Sherman, 42 Wallaby Way, Sydney")) # Works, """ ?\p{L}+""" matches "Hello", """ ?\p{L}+""" matches "world" etc.

['P', '.', ' Sherman', ',', ' ', '42', ' Wallaby', ' Way', ',', ' Sydney']


Let's actually go back to [GitHub - OpenAI/gpt-2/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py).<br>
We can see that the tokenizer is retrieved from the `encoder.json` file and the `vocab.bpe` file.<br>
- `encoder.json` holds the token-to-integer mapping.
- `vocab.bpe` holds the byte-pair encoding.

In [ ]:
# Retrieve the GPT-2 Tokenizer just like TikToken does
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

In [27]:
# This is equivalent to our `vocab`, Format is equal, just inverted: {idx: b'p0p1'} for us, {b'p0p1': idx} for them
with open('encoder.json', 'r') as f:
    encoder = json.load(f)

# This is equivalent to our `merges`, Format is equal for bpe_merges: {(child1, child2): idx}
with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()

bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]

In [28]:
print("Vocab Format is {b'p0p1': idx}:           ", list(encoder.items())[:5])
print("Merges Format is {(child1, child2): idx}: ", bpe_merges[:5])

Vocab Format is {b'p0p1': idx}:            [('!', 0), ('"', 1), ('#', 2), ('$', 3), ('%', 4)]
Merges Format is {(child1, child2): idx}:  [('Ġ', 't'), ('Ġ', 'a'), ('h', 'e'), ('i', 'n'), ('r', 'e')]


Concisely, while we implemented a trainable BPE tokenizer, OpenAI did so, too.<br><br>
However, they only share the trained result, not the training code that we ourselves implemented and which curiously enough is compatible with the file format OpenAI produces, too.<br>
And yet, the results of tokenization attained by us are different from what OpenAI attains, because of an architectural decision to add a second, closed-source, non-public encoder on top of the pseudo-public BPE tokenizer.

### Special Tokens

On top of guiding token derivation through regex-chunking, we can induce special tokens to be added to the vocabulary.<br>

In [29]:
print(len(encoder)) # 256 raw byte tokens + 50,000 merges + --> 1 special token <--
print(list(encoder.items())[len(encoder)-1])

50257
('<|endoftext|>', 50256)


> The special `<|endoftext|>` token is inserted in the training set to denote the end of one (contextually conntected) document and the beginning of another. The model should learn to inseart a conceptual 'break' as to not mix up the contents of the consecutive but maybe not related documents.

![](./img/Tiktoken_Vercel_2.png)

Think of special tokens as 'injected' tokens, not part of what BPE would call its vocabulary.<br><br>
Special tokens can can be injected and then trained. For example, the occurence of a special token might kick off a web search routine to enrich the context of the LLM.<br>
But this is also a crucial architecture for finetuning, like `<|im_start|>` for imaginary monologue<br>
or `<|fim_prefix|>`, `<|fim_middle|>`, `<|fim_suffix|>` according to [\[Bavarian et al., 2022\]](https://arxiv.org/abs/2207.14255) *(This is actually used in GPT-4)*.<br><br>
You can literally just go and fork the Tiktoken library and extend it by (consistently-indexed, mind you) custom special tokens. Tiktoken will handle the rest for you.


### Sentencepiece

The open-sourced [Sentencepiece](https://github.com/google/sentencepiece) library can perform both training and inference for BPE tokenization (amongst others).<br>

> Llama and Mistral models rely on Sentencepiece for tokenization.

With Tiktoken, we took our string data, decoded it to UTF-8, and then tokenized the Bytes from there.<br>
Sentencepiece runs BPE on the Unicode codepoints directly, not on the UTF-8 byte representations.<br>
It has an option `character_coverage` for what to do with codepoints that appear few times: It either maps them onto an UNK token, or, if `byte_fallback` is activated,<br>
encodes them with UTF-8 and then tokenizes the raw bytes instead.<br>
<br>
**Tiktoken is conceptually cleaner, while Sentencepiece is more efficient.**

In [30]:
import sentencepiece as spm

In [31]:
# Sentencepiece likes to work with files, so let's write our training data to a file:
with open("toy.txt", "w", encoding="utf-8") as f:
    # From the SentencePiece README (https://github.com/google/sentencepiece)
    f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing. This is not an official Google product.")

In [32]:
options = dict(
  # input
  input="toy.txt",
  input_format="text",
  # output
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  model_type="bpe", # BPE algorithm
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules (a different way to approach what tiktoken did through regex)
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special hard-coded tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)

In [33]:
sp = spm.SentencePieceProcessor()
sp.load("tok400.model")

# Inspect vocabulary off the model file
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [34]:
# Now, with the vocabulary sorted out, we can encode and decode text
ids = sp.encode("hello 안녕하세요")
print(ids) # This is the tokenized representation of the text

[359, 376, 360, 370, 370, 364, 359, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151]


In [35]:
print([sp.id_to_piece(idx) for idx in ids]) # This is the representation of what the token ids refer to

['▁', 'h', 'e', 'l', 'l', 'o', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>']


In the above example Sentencepiece encountered inputs that `vocab` does not account for.<br>
This would be problematic, if it weren't for `byte_fallback=True`.<br>
With this flag set, Sentencepiece encodes the input as UTF-8 and tokenizes the raw bytes as a fallback.

> If it weren't for `byte_fallback=True`, we'd get `['_', 'h', 'e', 'l', 'l', 'o', '_', '<unk>']`, because we never really saw the Korean characters in the training data, never mapped them to tokens, which would cause them to ruthlessly get replaced with the `<unk>` token.

While we're at it, why is there an extra `'_'` token in the front?<br>
It's caused by `add_dummy_prefix=True` in order to encode a mid-sentence `_hello` and a beginning-of-sentence `hello` equally.

The setup we justed walked through is really close to how Llama was trained.<br>
Actually, refer to [this issue](https://github.com/google/sentencepiece/issues/121) if you want the *exact* setup for Llama.

---

## Exercise Time

You can find an exercise to implement a BPE tokenizer in the lecture-accompanying [repository](https://github.com/karpathy/minbpe/blob/master/exercise.md).<br>Solving this here would not be a good fit for this notebook.<br><br>

---

## Looping Back: vocab_size

We already talked about this somewhat in the beginning of this chapter, but in the [last chapter](../N007%20-%20GPT%20From%20Scratch/N007%20-%20GPT.ipynb), we implemented [`gpt.py`](../N007%20-%20GPT%20From%20Scratch/gpt.py) in PyTorch.<br>
Our `vocab_size` was $65$. We represented each token by a vector of $65$ dimensions. Still, this was *character-level tokenization*.<br>
It becomes obvious that this approach scales poorly with a growing `vocab_size`, because within each of the $65$ vector representations,<br>
we note the likelihood for each of the $65$ tokens to follow the current token.

> The bigger this distribution becomes, the less expressive each vector becomes on what distinct token to favor for the next position.

In [ ]:
torch.manual_seed(1337) # for reproducibility

# Not really an LM at this stage, but we will get there...
class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Embedding the vocabulary
        # Every one of the vocab_size tokens is represented by a vector of size vocab_size
        self.embed = nn.Embedding(vocab_size, vocab_size) # 65 unique 65-dim vectors

    def forward(self, idx, targets):
        # idx is of shape (batch_size, block_size)
        # targets is of shape (batch_size, block_size)
        logits = self.embed(idx)
        return logits # Embed the input indices, shape is now (batch_size, block_size, vocab_size) (B, T, C)

What if we want to use an existing model and extend its vocabulary size? What about adding a special token?<br>
Well, both can be done, but the model's embedding matrix has to be retrained. This however can be done fairly well.<br>

## Conclusion

Honestly, modeling, reshaping and finetuning token vocabularies is a really interesting field of research, see [\[Mu et al., 2023\]](https://arxiv.org/abs/2304.08467).<br>
Beyond that, the question of vocabulary and tokenization increasingly drifts away from being exclusively a tool to model language.<br>
Papers like [\[Esser et al., 2020\]](https://arxiv.org/abs/2012.09841) for example investigate how to feed-in images as additional modalities to LLMs.<br>

> It seems that the field experiences a convergence in that the transformer is not to be touched, but the tokenization is to be extended. "Pretend this image is text, too." so to say.<br>
> This is what OpenAI SORA does, chunking image inputs efficiently into tokens and processing downstream with a transformer architecture.

Ok, to wrap things up, let's try to find answers to these earlier exclaimed issues:

- **Why can't the LLM spell?**
    - Characters are chunked up into differently sized tokens; The prompt `How many letters 'l' are in ".DefaultCellStyle"` gets various and false results.
- **Why can't the LLM do simple text manipulations like reversing a string?**
    - Again, characters are chunked up into differently sized tokens; This makes ChatGPT trip, but reversing `.D e f a u l t C e l l S t y l e` works for this reason.
- **Why can an LLM turn out to be worse with non-English languages?**
    - Lower representation in the training data leads to smaller (if any) tokenization of non-English characters. This bloats the Attention buffer. Bad.
- **Why can't an LLM produce simple arithmetic operations correctly?**
    - Number tokenization is the root cause for this. Or, more fittingly, [Integer tokenization is insane](https://www.beren.io/2023-02-04-Integer-tokenization-is-insane/)
- **Why could entering a special character like `<|endoftext|>` halt generation?**
    - It's one of those special tokens. It is interpreted very specifically, therefore. Don't view it as a token, but as a command
- **Why can an LLM have hiccups because of 'tailing whitespaces'?**
    - Tokens, e.g. for GPT, often follow the pattern `(space)(something)`, so adding a space makes GPT try to come up with something fitting to this pre-existing space too, which is rare and therefore 'pollutes' the context.
- **Why can an LLM break down when encountering capital letters within a word?**
    - An LLM has never seen this before; It's confused and really quickly diverts to predicting end-of-word tokens
- **Why may working with LLMs through YAML be better than through JSON?**
    - YAML contains less special characters, so it's less likely to trip the tokenizer
- **Why does LLM not actually mean end-to-end language modeling?**
    - We need to Tokenize to form a uniform and scalable basis for text representation
- **What on earth is going on with [`SolidGoldMagikarp`](https://www.lesswrong.com/posts/aPeJE8bSo6rAFoLqg/solidgoldmagikarp-plus-prompt-generation)?**
    - This absolutely broke LLMs like ChatGPT, essentially jailbreaking it. It seems the reddit user [SolidGoldMagikarp](https://reddit.com/u/SolidGoldMagikarp) posted so frequently that the tokenizer knows it from its training, but the LLM doesn't


> Eternal glory goes to those who can get rid of tokenization.


## References
References beyond Andrej Karpathy's lecture itself. Linked papers, blogposts, and other resources are listed here.

- [**Flashback: Character-Level Tokenization**](#flashback-character-level-tokenization)
- [**The Pitfalls of Tokenization**](#the-pitfalls-of-tokenization)
  - [Language Models are Unsupervised Multitask Learners \[Radford et al., 2019\]](https://insightcivic.s3.us-east-1.amazonaws.com/language-models.pdf)
  - [Tiktokenizer.vercel.app](https://tiktokenizer.vercel.app/)
- [**The Tokenizer Idea**](#the-tokenizer-idea)
  - [**Unicode**](#unicode)
    - [Nathan Reed - Programmer's Intro to Unicode](https://www.reedbeta.com/blog/programmers-intro-to-unicode/)
    - [UTF-8 Everywhere Manifesto](https://utf8everywhere.org/)
    - [MEGABYTE: Predicting Million-byte Sequences with Multiscale Transformers \[Yu et al., 2023\]](https://arxiv.org/abs/2305.07185)
- [**Byte-Pair Encoding**](#byte-pair-encoding)
  - [**Decoding**](#decoding)
  - [**Encoding**](#encoding)
- [**Going SOTA: Tokenizers in the Wild**](#going-sota-tokenizers-in-the-wild)
  - [**GPT Tokenizers**](#gpt-tokenizers)
    - [Language Models are Unsupervised Multitask Learners \[Radford et al., 2019\]](https://insightcivic.s3.us-east-1.amazonaws.com/language-models.pdf)
    - [Neural Machine Translation of Rare Words with Subword Units \[Sennrich et al., 2015\]](https://arxiv.org/abs/1508.07909)
    - [GitHub.com/openai/GPT-2](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
  - [**OpenAI TikToken**](#openai-tiktoken)
    - [GitHub.com/openai/tiktoken](https://github.com/openai/tiktoken)
  - [**Special Tokens**](#special-tokens)
    - [Efficient Training of Language Models to Fill in the Middle \[Bavarian et al., 2022\]](https://arxiv.org/abs/2207.14255)
  - [**Sentencepiece**](#sentencepiece)
    - [GitHub.com/google/sentencepiece](https://github.com/google/sentencepiece)
    - [GitHub.com/google/sentencepiece Issue #121](https://github.com/google/sentencepiece/issues/121)
- [**Exercise Time**](#exercise-time)
    - [GitHub.com/karpathy/minbpe](https://github.com/karpathy/minbpe)
- [**Looping Back: vocab_size**](#looping-back-vocab_size)
- [**Conclusion**](#conclusion)
    - [Learning to Compress Prompts with Gist Tokens \[Mu et al., 2023\]](https://arxiv.org/abs/2304.08467)
    - [Taming Transformers for High-Resolution Image Synthesis \[Esser et al., 2020\]](https://arxiv.org/abs/2012.09841)
    - [Beren Millidge: Integer tokenization is insane](https://www.beren.io/2023-02-04-Integer-tokenization-is-insane/)
    - [Jessica Rumbelow: SolidGoldMagikarp (plus, prompt generation)](https://www.lesswrong.com/posts/aPeJE8bSo6rAFoLqg/solidgoldmagikarp-plus-prompt-generation)

<center>Notebook by <a href="https://github.com/mk2112" target="_blank">mk2112</a>.</center>